# Learning 3: Illumination Modes in Microscopy

**Learning Objectives**:
By the end of this notebook, you will be able to:
1. Understand the difference between brightfield, darkfield, phase contrast, and DIC
2. Select appropriate illumination mode for sample type
3. Understand coherent vs incoherent imaging
4. Apply appropriate mode to transparent vs absorbing samples

**Estimated Runtime**: 20-30 minutes

**Prerequisites**: 
- Basic understanding of microscopy (see `quickstart_01_microscopy_basic.ipynb`)
- Understanding of resolution limits (see `learning_01_resolution_fundamentals.ipynb`)

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from prism.config.constants import um
from prism.core.instruments import create_instrument
from prism.scenarios import get_scenario_preset, list_scenario_presets
from prism.scenarios.microscopy import MicroscopeBuilder


# Set up plotting style
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 10

## Introduction: Why Different Illumination Modes?

Different samples require different imaging approaches:

| Sample Type | Challenge | Solution |
|-------------|-----------|----------|
| Stained cells | Absorbing | Brightfield |
| Live cells | Transparent, no contrast | Phase Contrast |
| Small particles | Low signal | Darkfield |
| Textured surfaces | Need 3D relief | DIC |

Let's explore each mode and understand when to use them!

---

## Section 1: Brightfield Imaging

**Principle**: Light passes through or reflects from the sample. Contrast comes from light absorption.

**Best for**: 
- Stained samples (H&E, immunohistochemistry)
- Naturally colored samples
- Absorbing structures

**Limitation**: Poor contrast for transparent/unstained samples

In [ ]:
# Load brightfield microscope preset
scenario_bf = get_scenario_preset("microscope_40x_air")

print("Brightfield Microscope Configuration:")
print(f"  Objective: {scenario_bf.objective_spec}")
print(f"  Illumination Mode: {scenario_bf.illumination_mode}")
print(f"  Resolution: {scenario_bf.lateral_resolution_nm:.0f} nm")
print(f"  NA: {scenario_bf._obj.numerical_aperture}")

In [ ]:
# Create microscope instrument
microscope_bf = create_instrument(scenario_bf.to_instrument_config())

# Get the expected grid size from the scenario
n_pixels = scenario_bf.n_pixels  # Use microscope's expected size (1024)

# Create two test samples:
# 1. Absorbing sample (like stained cells) - amplitude modulation
# 2. Transparent sample (like live cells) - phase-only modulation

# Calculate physical field size based on microscope FOV
field_size = scenario_bf.field_of_view_um * 1e-6  # Convert µm to m
pixel_size = field_size / n_pixels

# Create coordinate grids
x = torch.linspace(-field_size / 2, field_size / 2, n_pixels)
y = torch.linspace(-field_size / 2, field_size / 2, n_pixels)
X, Y = torch.meshgrid(x, y, indexing="xy")

# Absorbing sample: circles with varying absorption (amplitude)
absorbing_sample = torch.ones(n_pixels, n_pixels)
# Add absorbing circles (like stained organelles)
for cx, cy, r, absorption in [
    (-20 * um, -20 * um, 10 * um, 0.3),  # Large dark circle
    (15 * um, 15 * um, 8 * um, 0.5),  # Medium gray circle
    (-10 * um, 18 * um, 6 * um, 0.7),  # Small light circle
    (18 * um, -12 * um, 7 * um, 0.4),  # Another circle
]:
    mask = ((X - cx) ** 2 + (Y - cy) ** 2) < r**2
    absorbing_sample[mask] = absorption

# Transparent sample: phase-only circles (no amplitude change)
# Phase objects are invisible in brightfield!
transparent_amplitude = torch.ones(n_pixels, n_pixels)  # Uniform amplitude
transparent_phase = torch.zeros(n_pixels, n_pixels)  # Phase variations
for cx, cy, r, phase_shift in [
    (-20 * um, -20 * um, 10 * um, np.pi / 4),  # Large phase shift
    (15 * um, 15 * um, 8 * um, np.pi / 3),  # Medium phase shift
    (-10 * um, 18 * um, 6 * um, np.pi / 6),  # Small phase shift
    (18 * um, -12 * um, 7 * um, np.pi / 2),  # Another phase shift
]:
    mask = ((X - cx) ** 2 + (Y - cy) ** 2) < r**2
    transparent_phase[mask] = phase_shift

# Create complex field for transparent sample (phase only)
transparent_sample = transparent_amplitude * torch.exp(1j * transparent_phase)

print(f"Created two test samples: {n_pixels}x{n_pixels} pixels")
print(f"Field size: {field_size * 1e6:.1f} µm")
print(f"Pixel size: {pixel_size * 1e9:.1f} nm")

In [ ]:
# Simulate brightfield imaging of both samples
# Add batch and channel dimensions
absorbing_input = absorbing_sample.unsqueeze(0).unsqueeze(0)
transparent_input = transparent_sample.unsqueeze(0).unsqueeze(0)

with torch.no_grad():
    bf_absorbing = microscope_bf.forward(absorbing_input, illumination_mode="brightfield")
    bf_transparent = microscope_bf.forward(transparent_input, illumination_mode="brightfield")

# Remove batch/channel dims
bf_absorbing_img = bf_absorbing.squeeze().cpu().numpy()
bf_transparent_img = bf_transparent.squeeze().cpu().real.numpy()

In [ ]:
# Visualize brightfield results
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
extent = [-field_size * 1e6 / 2, field_size * 1e6 / 2, -field_size * 1e6 / 2, field_size * 1e6 / 2]

# Ground truth - absorbing
axes[0, 0].imshow(absorbing_sample.numpy(), cmap="gray", extent=extent, vmin=0, vmax=1)
axes[0, 0].set_title("Absorbing Sample (Ground Truth)\nLike stained cells")
axes[0, 0].set_xlabel("Position (µm)")
axes[0, 0].set_ylabel("Position (µm)")

# Brightfield - absorbing
axes[0, 1].imshow(bf_absorbing_img, cmap="gray", extent=extent)
axes[0, 1].set_title("Brightfield: Absorbing Sample\n✅ Good contrast!")
axes[0, 1].set_xlabel("Position (µm)")
axes[0, 1].set_ylabel("Position (µm)")

# Ground truth - transparent (show phase)
axes[1, 0].imshow(transparent_phase.numpy(), cmap="twilight", extent=extent)
axes[1, 0].set_title("Transparent Sample (Phase Map)\nLike live unstained cells")
axes[1, 0].set_xlabel("Position (µm)")
axes[1, 0].set_ylabel("Position (µm)")

# Brightfield - transparent
axes[1, 1].imshow(bf_transparent_img, cmap="gray", extent=extent)
axes[1, 1].set_title("Brightfield: Transparent Sample\n❌ Almost invisible!")
axes[1, 1].set_xlabel("Position (µm)")
axes[1, 1].set_ylabel("Position (µm)")

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight:")
print("Brightfield works well for ABSORBING samples (top row)")
print("but fails for TRANSPARENT samples (bottom row) - they're nearly invisible!")

---

## Section 2: Phase Contrast Microscopy

**Principle**: Converts invisible phase shifts into visible intensity changes.

**How it works**:
1. Light passing through transparent samples experiences phase shifts (optical path length changes)
2. A phase ring in the objective adds λ/4 delay to direct (unscattered) light
3. Interference between direct and diffracted light creates intensity contrast

**Best for**:
- Live cells (unstained)
- Transparent biological samples
- Observing cell dynamics without staining

In [ ]:
# Load phase contrast preset
scenario_phase = get_scenario_preset("microscope_40x_phase")

print("Phase Contrast Microscope Configuration:")
print(f"  Objective: {scenario_phase.objective_spec}")
print(f"  Illumination Mode: {scenario_phase.illumination_mode}")
print(f"  Resolution: {scenario_phase.lateral_resolution_nm:.0f} nm")

In [ ]:
# Create phase contrast microscope
microscope_phase = create_instrument(scenario_phase.to_instrument_config())

# Image the transparent sample with phase contrast
with torch.no_grad():
    phase_transparent = microscope_phase.forward(transparent_input, illumination_mode="phase")

phase_transparent_img = phase_transparent.squeeze().cpu().real.numpy()

In [ ]:
# Compare brightfield vs phase contrast for transparent sample
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Ground truth phase
im0 = axes[0].imshow(transparent_phase.numpy(), cmap="twilight", extent=extent)
axes[0].set_title("Ground Truth: Phase Shifts\n(invisible to human eye)")
axes[0].set_xlabel("Position (µm)")
axes[0].set_ylabel("Position (µm)")
plt.colorbar(im0, ax=axes[0], label="Phase (rad)")

# Brightfield (fails)
axes[1].imshow(bf_transparent_img, cmap="gray", extent=extent)
axes[1].set_title("Brightfield\n❌ No contrast")
axes[1].set_xlabel("Position (µm)")
axes[1].set_ylabel("Position (µm)")

# Phase contrast (works!)
axes[2].imshow(phase_transparent_img, cmap="gray", extent=extent)
axes[2].set_title("Phase Contrast\n✅ Phase → Intensity!")
axes[2].set_xlabel("Position (µm)")
axes[2].set_ylabel("Position (µm)")

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight:")
print("Phase contrast converts PHASE information into INTENSITY contrast.")
print("This is why Frits Zernike won the Nobel Prize in Physics (1953)!")

In [ ]:
# Visualize how phase contrast works: the phase ring
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Compute PSFs for comparison
psf_bf = microscope_bf.compute_psf(illumination_mode="brightfield")
psf_phase = microscope_phase.compute_psf(illumination_mode="phase")

# Shift to center for visualization
psf_bf_centered = torch.fft.fftshift(psf_bf).cpu().numpy()
psf_phase_centered = torch.fft.fftshift(psf_phase).cpu().numpy()

# Show PSFs
center = psf_bf_centered.shape[0] // 2
crop = 30

axes[0].imshow(
    psf_bf_centered[center - crop : center + crop, center - crop : center + crop], cmap="hot"
)
axes[0].set_title("Brightfield PSF\n(Standard Airy disk)")
axes[0].axis("off")

axes[1].imshow(
    psf_phase_centered[center - crop : center + crop, center - crop : center + crop], cmap="hot"
)
axes[1].set_title("Phase Contrast PSF\n(Modified by phase ring)")
axes[1].axis("off")

plt.tight_layout()
plt.show()

---

## Section 3: Darkfield and DIC Microscopy

### Darkfield Microscopy

**Principle**: Only scattered light reaches the detector. Direct light is blocked.

**How it works**:
1. Illumination comes from a hollow cone (annular aperture)
2. Direct light misses the objective's acceptance cone
3. Only light scattered by the sample enters the objective

**Best for**:
- Small particles and bacteria
- Edge detection
- Low-contrast samples where scattering is present

---

### DIC (Differential Interference Contrast)

**Principle**: Creates pseudo-3D relief by detecting phase gradients.

**How it works**:
1. Polarized light is split into two beams with slight lateral displacement (shear)
2. Beams pass through sample at slightly different positions
3. Interference reveals optical path *differences* → directional gradients

**Best for**:
- Thick transparent samples
- Surface topology
- 3D-like visualization without fluorescence

In [ ]:
# Create darkfield configuration using MicroscopeBuilder
scenario_darkfield = (
    MicroscopeBuilder()
    .objective("40x_0.9NA_air")
    .illumination("darkfield")
    .wavelength_nm(550)
    .name("40x Darkfield")
    .build()
)

# Load DIC preset
scenario_dic = get_scenario_preset("microscope_40x_dic")

print("Darkfield Configuration:")
print(f"  Mode: {scenario_darkfield.illumination_mode}")

print("\nDIC Configuration:")
print(f"  Mode: {scenario_dic.illumination_mode}")

In [ ]:
# Create microscopes
microscope_darkfield = create_instrument(scenario_darkfield.to_instrument_config())
microscope_dic = create_instrument(scenario_dic.to_instrument_config())

# Create a sample with small scattering features (good for darkfield)
# Small particles scattered throughout
scattering_sample = torch.ones(n_pixels, n_pixels, dtype=torch.complex64)
np.random.seed(42)
n_particles = 80
for _ in range(n_particles):
    px = np.random.randint(100, n_pixels - 100)
    py = np.random.randint(100, n_pixels - 100)
    radius = np.random.randint(3, 10)  # Small particles
    for dx in range(-radius, radius + 1):
        for dy in range(-radius, radius + 1):
            if dx**2 + dy**2 <= radius**2:
                if 0 <= px + dx < n_pixels and 0 <= py + dy < n_pixels:
                    # Add both absorption and phase shift
                    scattering_sample[py + dy, px + dx] = 0.5 * torch.exp(
                        1j * torch.tensor(np.pi / 4)
                    )

scattering_input = scattering_sample.unsqueeze(0).unsqueeze(0)

In [ ]:
# Image with all four modes
with torch.no_grad():
    img_bf_scatter = microscope_bf.forward(scattering_input, illumination_mode="brightfield")
    img_df_scatter = microscope_darkfield.forward(scattering_input, illumination_mode="darkfield")
    img_phase_scatter = microscope_phase.forward(scattering_input, illumination_mode="phase")
    img_dic_scatter = microscope_dic.forward(scattering_input, illumination_mode="dic")

# Extract images
bf_scatter_img = img_bf_scatter.squeeze().cpu().real.numpy()
df_scatter_img = img_df_scatter.squeeze().cpu().real.numpy()
phase_scatter_img = img_phase_scatter.squeeze().cpu().real.numpy()
dic_scatter_img = img_dic_scatter.squeeze().cpu().real.numpy()

In [ ]:
# Compare all four illumination modes on scattering sample
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Brightfield
axes[0, 0].imshow(bf_scatter_img, cmap="gray", extent=extent)
axes[0, 0].set_title("Brightfield\nGood overall view, moderate contrast")
axes[0, 0].set_xlabel("Position (µm)")
axes[0, 0].set_ylabel("Position (µm)")

# Darkfield
axes[0, 1].imshow(df_scatter_img, cmap="gray", extent=extent)
axes[0, 1].set_title("Darkfield\nScattered light only - bright particles on dark background")
axes[0, 1].set_xlabel("Position (µm)")
axes[0, 1].set_ylabel("Position (µm)")

# Phase Contrast
axes[1, 0].imshow(phase_scatter_img, cmap="gray", extent=extent)
axes[1, 0].set_title("Phase Contrast\nEnhanced contrast for transparent features")
axes[1, 0].set_xlabel("Position (µm)")
axes[1, 0].set_ylabel("Position (µm)")

# DIC
axes[1, 1].imshow(dic_scatter_img, cmap="gray", extent=extent)
axes[1, 1].set_title("DIC\nPseudo-3D relief showing edges/gradients")
axes[1, 1].set_xlabel("Position (µm)")
axes[1, 1].set_ylabel("Position (µm)")

plt.tight_layout()
plt.show()

In [ ]:
# Create a textured surface sample (ideal for DIC)
# Simulate a surface with height variations → phase gradients
textured_phase = torch.zeros(n_pixels, n_pixels)

# Add sinusoidal surface texture
freq = 15  # cycles across field
textured_phase = 0.5 * torch.sin(2 * np.pi * freq * X / field_size)
textured_phase += 0.3 * torch.sin(2 * np.pi * freq * 1.5 * Y / field_size)

# Add some localized features
for cx, cy, r in [(-20 * um, 8 * um, 10 * um), (15 * um, -15 * um, 8 * um)]:
    dist = torch.sqrt((X - cx) ** 2 + (Y - cy) ** 2)
    bump = torch.exp(-(dist**2) / (2 * (r / 2) ** 2)) * np.pi / 3
    textured_phase += bump

textured_sample = torch.exp(1j * textured_phase)
textured_input = textured_sample.unsqueeze(0).unsqueeze(0)

# Image with all modes
with torch.no_grad():
    img_bf_tex = microscope_bf.forward(textured_input, illumination_mode="brightfield")
    img_df_tex = microscope_darkfield.forward(textured_input, illumination_mode="darkfield")
    img_phase_tex = microscope_phase.forward(textured_input, illumination_mode="phase")
    img_dic_tex = microscope_dic.forward(textured_input, illumination_mode="dic")

In [ ]:
# Compare modes on textured surface
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Ground truth phase (surface height)
im = axes[0, 0].imshow(textured_phase.numpy(), cmap="terrain", extent=extent)
axes[0, 0].set_title("Ground Truth: Surface Height\n(Phase map)")
axes[0, 0].set_xlabel("Position (µm)")
axes[0, 0].set_ylabel("Position (µm)")
plt.colorbar(im, ax=axes[0, 0], label="Height (rad)")

# Brightfield
axes[0, 1].imshow(img_bf_tex.squeeze().cpu().real.numpy(), cmap="gray", extent=extent)
axes[0, 1].set_title("Brightfield\n❌ Texture nearly invisible")
axes[0, 1].set_xlabel("Position (µm)")

# Darkfield
axes[0, 2].imshow(img_df_tex.squeeze().cpu().real.numpy(), cmap="gray", extent=extent)
axes[0, 2].set_title("Darkfield\nShows scattering at edges")
axes[0, 2].set_xlabel("Position (µm)")

# Empty for layout
axes[1, 0].axis("off")

# Phase Contrast
axes[1, 1].imshow(img_phase_tex.squeeze().cpu().real.numpy(), cmap="gray", extent=extent)
axes[1, 1].set_title("Phase Contrast\nShows phase variations")
axes[1, 1].set_xlabel("Position (µm)")
axes[1, 1].set_ylabel("Position (µm)")

# DIC - best for textured surfaces!
axes[1, 2].imshow(img_dic_tex.squeeze().cpu().real.numpy(), cmap="gray", extent=extent)
axes[1, 2].set_title("DIC\n✅ Best for surface texture!")
axes[1, 2].set_xlabel("Position (µm)")

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight:")
print("DIC reveals surface GRADIENTS, creating a pseudo-3D relief effect.")
print("It's ideal for visualizing texture and topography!")

---

## Section 4: Mode Selection Guide

### Decision Tree: Which Mode Should I Use?

```
                    Is the sample transparent?
                           /        \
                         NO          YES
                         |            |
                    BRIGHTFIELD    What do you need?
                    (absorbing)         /    |    \
                                       /     |     \
                              Phase info  3D relief  High contrast
                              for cells   for texture for particles
                                  |          |           |
                            PHASE CONTRAST   DIC      DARKFIELD
```

In [ ]:
# Interactive mode selection guide
def recommend_illumination_mode(sample_type: str) -> str:
    """Recommend best illumination mode for sample type.

    Args:
        sample_type: Description of sample

    Returns:
        Recommended illumination mode with explanation
    """
    sample_lower = sample_type.lower()

    if any(word in sample_lower for word in ["stain", "dye", "color", "absorb", "histology"]):
        return "BRIGHTFIELD - Stained/absorbing samples have inherent contrast"

    elif any(word in sample_lower for word in ["live", "unstained", "transparent", "cell culture"]):
        return "PHASE CONTRAST - Converts invisible phase shifts to visible contrast"

    elif any(word in sample_lower for word in ["particle", "bacteria", "nano", "small", "scatter"]):
        return "DARKFIELD - High contrast for small scattering objects"

    elif any(word in sample_lower for word in ["surface", "texture", "relief", "3d", "topograph"]):
        return "DIC - Shows gradients and surface topology as 3D relief"

    else:
        return "BRIGHTFIELD - Default mode, try others if contrast is poor"


# Test samples
test_samples = [
    "H&E stained tissue section",
    "Live HeLa cells in culture",
    "Gold nanoparticles in solution",
    "Textured polymer surface",
    "Unstained transparent organoids",
    "Bacteria in water sample",
]

print("Mode Selection Guide:")
print("=" * 70)
for sample in test_samples:
    recommendation = recommend_illumination_mode(sample)
    print(f"\n📋 Sample: {sample}")
    print(f"   → {recommendation}")

In [ ]:
# Summary comparison table
print("\n" + "=" * 80)
print("ILLUMINATION MODES COMPARISON")
print("=" * 80)

comparison_data = [
    ["Mode", "Best For", "Mechanism", "Background"],
    ["-" * 15, "-" * 25, "-" * 25, "-" * 10],
    ["Brightfield", "Stained/absorbing samples", "Absorption contrast", "Bright"],
    ["Phase Contrast", "Live cells, transparent", "Phase → intensity", "Gray halo"],
    ["Darkfield", "Small particles, edges", "Scattered light only", "Dark"],
    ["DIC", "Surfaces, thick samples", "Phase gradients", "Pseudo-3D"],
]

for row in comparison_data:
    print(f"{row[0]:15s} | {row[1]:25s} | {row[2]:25s} | {row[3]:10s}")

In [ ]:
# Final comprehensive visualization: All 4 modes on transparent sample
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

modes = [
    ("brightfield", microscope_bf, "Brightfield\n(Direct transmission)"),
    ("darkfield", microscope_darkfield, "Darkfield\n(Scattered light only)"),
    ("phase", microscope_phase, "Phase Contrast\n(Phase → Intensity)"),
    ("dic", microscope_dic, "DIC\n(Phase gradients)"),
]

for ax, (mode, microscope, title) in zip(axes.flat, modes):
    with torch.no_grad():
        img = microscope.forward(transparent_input, illumination_mode=mode)

    ax.imshow(img.squeeze().cpu().real.numpy(), cmap="gray", extent=extent)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Position (µm)")
    ax.set_ylabel("Position (µm)")

plt.suptitle("Transparent Phase Sample: Four Illumination Modes Compared", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## Assessment Questions

Test your understanding of illumination modes:

### Question 1
**Which illumination mode would you choose for imaging unstained live cells?**

<details>
<summary>Click for answer</summary>

**Phase Contrast** - Live cells are transparent and don't absorb light significantly. Phase contrast converts the phase shifts (from refractive index variations within cells) into visible intensity differences.
</details>

### Question 2
**Why does brightfield fail for transparent samples?**

<details>
<summary>Click for answer</summary>

Brightfield imaging relies on **amplitude (absorption) contrast**. Transparent samples don't absorb light - they only shift the phase of light passing through them. Since human eyes and cameras detect intensity (not phase), transparent samples appear invisible in brightfield.
</details>

### Question 3
**When should you use darkfield instead of phase contrast?**

<details>
<summary>Click for answer</summary>

Use **darkfield** when:
- Imaging small particles or bacteria (high contrast against dark background)
- Looking for edges and boundaries
- Sample scatters light but doesn't have significant phase variations

Use **phase contrast** when:
- Imaging cells with internal phase structure
- Need to see internal details of transparent objects
- Quantitative phase information is desired
</details>

---

## Summary

In this notebook, we learned:

1. ✅ **Brightfield**: Best for absorbing/stained samples, fails for transparent samples
2. ✅ **Phase Contrast**: Converts invisible phase shifts to visible contrast - perfect for live cells
3. ✅ **Darkfield**: Shows only scattered light - ideal for small particles and edges
4. ✅ **DIC**: Reveals phase gradients as pseudo-3D relief - great for textured surfaces

**Key Takeaways**:
- Match illumination mode to sample type for optimal contrast
- Transparent samples require phase-sensitive techniques
- Different modes reveal different sample properties
- SPIDS supports all major illumination modes!

---

## Reference Documentation

For detailed parameter lookup and formulas:

- **[Microscopy Parameters](../../docs/references/microscopy_parameters.md)** - Objective specs, illumination optics
- **[Optical Resolution Limits](../../docs/references/optical_resolution_limits.md)** - Abbe limit, contrast transfer
- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 9 microscope presets including phase and DIC

---

## Next Steps

- **[Learning 1: Resolution Fundamentals](learning_01_resolution_fundamentals.ipynb)**: Review Abbe limit and PSFs
- **[Learning 2: Resolution Validation](learning_02_resolution_validation.ipynb)**: USAF-1951 testing
- **[06_microscope_reconstruction.py](../../examples/python_api/06_microscope_reconstruction.py)**: Full SPIDS reconstruction pipeline

In [ ]:
# List all available microscope presets
print("\nAvailable Microscope Presets:")
print("=" * 50)
for preset in list_scenario_presets("microscope"):
    scenario = get_scenario_preset(preset)
    print(f"  {preset:35s} - {scenario.illumination_mode}")